In [16]:
import requests
from config import API_key
import pandas as pd
from pprint import pprint
from json.decoder import JSONDecodeError
import warnings
warnings.filterwarnings('ignore')

In [3]:
summary = "Data/summary.csv"
summary = pd.read_csv(summary)
summary

,Unnamed: 0,Date,City,State,Country,Shape,Duration,Summary,Posted,Images,ReportNum
0,0,7/31/22 21:59,West Lafayette,OH,USA,Light,4 minutes,Bright circle light moving through sky.,9/9/22,Yes,1
1,1,7/31/22 21:45,Burr Ridge,IL,USA,Other,NaN,"Strange red, blue and black craft in the sky a...",9/9/22,Yes,2
2,2,7/31/22 21:15,Conway,SC,USA,Circle,Less than a minute,Round bright white light go in a straight line...,9/9/22,NaN,3
3,3,7/31/22 21:11,Keyser,WV,USA,Light,2 minutes,Saw an orb that interacted with me.,9/9/22,NaN,4
4,4,7/31/22 20:43,gressham,OR,USA,Unknown,less than a minute,A single soft red light that pulsed three time...,9/9/22,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...
2126,2126,12/1/22 18:00,Snowflake,AZ,USA,Triangle,2 minutes,I grew up on a 40 acre parcel of land with thr...,12/22/22,NaN,2127
2127,2127,12/1/22 15:33,NaN,NaN,USA,Cross,NaN,I observed some type of phenomenon that I was ...,12/22/22,Yes,2128
2128,2128,12/1/22 14:10,El paso,TX,USA,Oval,Ten minutes,It appeared out of nowhere & slowly moved acro...,12/22/22,NaN,2129
2129,2129,12/1/22 06:20,CHARLESTOWN,IN,USA,Light,2 to 3 minutes,Bright object was seen arriving and silently h...,12/22/22,Yes,2130


In [4]:
# rename and drop the first column
summary.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
summary.drop(["a"], axis=1, inplace=True)

In [5]:
# reducing the df
city_df = summary[['City', 'State', 'ReportNum']]

In [6]:
# replace with space
city_df.fillna("")

,City,State,ReportNum
0,West Lafayette,OH,1
1,Burr Ridge,IL,2
2,Conway,SC,3
3,Keyser,WV,4
4,gressham,OR,5
...,...,...,...
2126,Snowflake,AZ,2127
2127,,,2128
2128,El paso,TX,2129
2129,CHARLESTOWN,IN,2130


In [17]:
# concatenate the city and state colum to pass in the API
city_df["Location"] = city_df[['City', 'State']].apply(lambda x: ', '.join(x[x.notnull()]), axis=1)

In [8]:
city_df

,City,State,ReportNum,Location
0,West Lafayette,OH,1,"West Lafayette, OH"
1,Burr Ridge,IL,2,"Burr Ridge, IL"
2,Conway,SC,3,"Conway, SC"
3,Keyser,WV,4,"Keyser, WV"
4,gressham,OR,5,"gressham, OR"
...,...,...,...,...
2126,Snowflake,AZ,2127,"Snowflake, AZ"
2127,NaN,NaN,2128,
2128,El paso,TX,2129,"El paso, TX"
2129,CHARLESTOWN,IN,2130,"CHARLESTOWN, IN"


In [18]:
# create new new columns for lat and lng
city_df["Latitude"]=""
city_df["Longitude"]=""
city_df

,City,State,ReportNum,Location,Latitude,Longitude
0,West Lafayette,OH,1,"West Lafayette, OH",,
1,Burr Ridge,IL,2,"Burr Ridge, IL",,
2,Conway,SC,3,"Conway, SC",,
3,Keyser,WV,4,"Keyser, WV",,
4,gressham,OR,5,"gressham, OR",,
...,...,...,...,...,...,...
2126,Snowflake,AZ,2127,"Snowflake, AZ",,
2127,NaN,NaN,2128,,,
2128,El paso,TX,2129,"El paso, TX",,
2129,CHARLESTOWN,IN,2130,"CHARLESTOWN, IN",,


In [11]:
# loop through the df 
for index,row in city_df.iterrows():
    city = row["Location"]
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=" 
    print(f"Processing Record for {city}")
    #     get the response   
    # Use try and except to skip the missing data
    try: 
        query_url = f"{url}{city}&key={API_key}"
        response = requests.get(query_url).json()
#         pprint(response)
        city_df.loc[index,"Latitude"] = response['results'][0]['geometry']['location']['lat']
        city_df.loc[index,"Longitude"]= response['results'][0]['geometry']['location']['lng']
    except(KeyError, IndexError, JSONDecodeError):
        print(f"City not found. Skipping...")
    except requests.ConnectionError:
        print("ConnectionError...")
       

Processing Record for West Lafayette, OH


C:\Users\13135\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Processing Record for Burr Ridge, IL
Processing Record for Conway, SC
Processing Record for Keyser, WV
Processing Record for gressham, OR
Processing Record for Eddyville, KY
Processing Record for Carpinteria, CA
Processing Record for Tacoma, WA
Processing Record for Charlotte, NC
Processing Record for Burnet, TX
Processing Record for Seattle, WA
Processing Record for Naples, FL
Processing Record for Tacoma, WA
Processing Record for Flemington, NJ
Processing Record for Fremont, CA
Processing Record for Bella Vista, AR
Processing Record for Dorchester, MA
Processing Record for Dalton Gardens, ID
Processing Record for Philadelphia, PA
Processing Record for Harrisburg, PA
Processing Record for Wexford, PA
Processing Record for Newark, NJ
Processing Record for Hicksville, NY
Processing Record for Chenoa
Processing Record for River Forest, IL
Processing Record for Eudora, KS
Processing Record for North Dartmouth, MA
Processing Record for Randolph, VT
Processing Record for Maupin, OR
Processi

Processing Record for Memphis, TN
Processing Record for Plymouth, VT
Processing Record for East Hartford, CT
Processing Record for Bowling Green, KY
Processing Record for Carson City
Processing Record for Gorham, ME
Processing Record for Seattle, WA
Processing Record for Torrance, CA
Processing Record for West Melbourne, FL
Processing Record for Hollywood, FL
Processing Record for Jupiter, FL
Processing Record for Marshall, TX
Processing Record for Phoenix, AZ
Processing Record for Norwalk, CT
Processing Record for Smith Mills, KY
Processing Record for YPSILANTI, MI
Processing Record for Fripp Island, SC
Processing Record for Moffat, CO
Processing Record for Morganton, NC
Processing Record for Memphis, TN
Processing Record for New Smyrna Beach, FL
Processing Record for Jefferson City, TN
Processing Record for Garland, TX
Processing Record for Ewing, NJ
Processing Record for Huntley, MT
Processing Record for Hazel Green, WI
Processing Record for Smyrna, TN
Processing Record for Dayton, 

Processing Record for Waukesha, WI
Processing Record for Geyserville, CA
Processing Record for Mustang, OK
Processing Record for Pittsburgh, PA
Processing Record for Cairo, KY
Processing Record for Colorado city, TX
Processing Record for Austin, TX
Processing Record for San Diego, CA
Processing Record for Littleton (Denver metro area), CO
Processing Record for Toledo, OH
Processing Record for Pendleton, OR
Processing Record for Lawrenceburg, KY
Processing Record for John Day, OR
Processing Record for Eagle Mountain, UT
Processing Record for Stanford, KY
Processing Record for Miami, FL
Processing Record for Birmingham, AL
Processing Record for Lower Lewis Falls Camp Ground, WA
Processing Record for New Albany, OH
Processing Record for Tyler, TX
Processing Record for Naples, NY
Processing Record for Klamath Falls, OR
Processing Record for Rancho Cucamonga, CA
Processing Record for Kalamazoo, MI
Processing Record for Dallas, TX
Processing Record for Colorado Springs, CO
Processing Record 

...

Processing Record for Loranger, LA
Processing Record for Thornton, CO
Processing Record for Hwy 71 between maryville - saint joeseph, MO
Processing Record for Luttrell-manerdville, TN
Processing Record for Apopka, FL
Processing Record for Milan, MI
Processing Record for Mansfield, MA
Processing Record for South Dennis, MA
Processing Record for Ellabell, GA
Processing Record for Berwyn, PA
Processing Record for Mt Charleston, NV
Processing Record for Chico, CA
Processing Record for Gulf Shores, AL
Processing Record for Baldwin, FL
Processing Record for Lake Tapps WA 98391
Processing Record for Duluth, GA
Processing Record for Corydon, IN
Processing Record for Tamarac, FL
Processing Record for Largo, FL
Processing Record for Pawley's Island, SC
Processing Record for Duluth, MN
Processing Record for Myakka City, FL
Processing Record for Aiken, SC
Processing Record for Chester Springs, PA
Processing Record for Walnut Creek, CA
Processing Record for Dover, DE
Processing Record for Mesa, AZ


Processing Record for Altmar, NY
Processing Record for Austin, TX
Processing Record for Melrose, MA
Processing Record for Port charlotte, FL
Processing Record for Lancaster, PA
Processing Record for Florence, KY
Processing Record for Yucaipa, CA
Processing Record for Oswego, IL
Processing Record for 
City not found. Skipping...
Processing Record for Denver, CO
Processing Record for Denver, CO
Processing Record for Tuscola, IL
Processing Record for Daytona Beach, FL
Processing Record for Amarillo, TX
Processing Record for Santa Clara, CA
Processing Record for Springfield, IL
Processing Record for SUSANVILLE, CA
Processing Record for Aurora, IL
Processing Record for Rutherfordton, NC
Processing Record for Montgomery, OH
Processing Record for Wildwood, FL
Processing Record for Birmingham, AL
Processing Record for Las Cruces, NM
Processing Record for Ashburn, GA
Processing Record for osage beach, MO
Processing Record for DUGSPUR, VA
Processing Record for Lexington, KY
Processing Record for

Processing Record for Wellington, FL
Processing Record for West Palm Beach, FL
Processing Record for Jersey City, NJ
Processing Record for Glendale, AZ
Processing Record for Goulds, FL
Processing Record for Piqua, OH
Processing Record for Spangle, WA
Processing Record for Brandon, MS
Processing Record for Rural area, TN
City not found. Skipping...
Processing Record for Clemons, NY
Processing Record for Foothill Ranch, CA
Processing Record for Carrollton, OH
Processing Record for Bedminster, NJ
Processing Record for Oklahoma City, OK
Processing Record for Dublin, VA
Processing Record for Coquille, OR
Processing Record for Kenner, LA
Processing Record for Fayetteville, AR
Processing Record for Republic, MO
Processing Record for Lynchburg, VA
Processing Record for Memphis, TN
Processing Record for Manchester, OH
Processing Record for El Dorado Hills
Processing Record for Lynnwood, WA
Processing Record for Rogers, AR
Processing Record for Wells, ME
Processing Record for Guilford, CT
Proces

Processing Record for 
City not found. Skipping...
Processing Record for San Antonio, TX
Processing Record for Gaithersburg, MD
Processing Record for Flint, MI
Processing Record for Hibbing or Kelly Lake, MN
Processing Record for Fisherville, KY
Processing Record for NORTH HOLLYWOOD, CA
Processing Record for Sun city, AZ
Processing Record for Orcutt, CA
Processing Record for Maplewood, MN
Processing Record for Holly Hill, SC
Processing Record for Winston Salem, NC
Processing Record for Brandywine, MD
Processing Record for Eugene, OR
Processing Record for Chicago, IL
Processing Record for bellgrade, MT
Processing Record for Fulks Run, VA
Processing Record for St Louis, MO
Processing Record for Mount Horeb, WI
Processing Record for Payson, AZ
Processing Record for Fallbrook, CA
Processing Record for Northridge, CA
Processing Record for The Villages, FL
Processing Record for Leavenworth, WA
Processing Record for West Hollywood, CA
Processing Record for Vestavia Hills, AL
Processing Record

Processing Record for Crossville, TN
Processing Record for Bowie, MD
Processing Record for Bradford, TN
Processing Record for Eureka Springs, AR
Processing Record for Eureka Springs, AR
Processing Record for Delray Beach, FL
Processing Record for Springfield, IL
Processing Record for WA
Processing Record for Tupelo, MS
Processing Record for Halfmoon bay, CA
Processing Record for Oklahoma City, OK
Processing Record for Galva, IA
Processing Record for Riverside, CA
Processing Record for Sterlng, CO
Processing Record for Westcliffe, CO
Processing Record for Millen, GA
Processing Record for Flushing, NY
Processing Record for Overland Park, KS
Processing Record for Cape Coral, FL
Processing Record for Belhaven, NC
Processing Record for NC
Processing Record for Franklin, NC
Processing Record for San Bernardino, CA
Processing Record for Santa Barbara, CA
Processing Record for Conway, NH
Processing Record for Somersworth
Processing Record for Flower mound, TX
Processing Record for Greenwood, I

In [12]:
city_df.head(150)
reduced_df = city_df[["ReportNum","Location","Latitude","Longitude"]]
reduced_df


,ReportNum,Location,Latitude,Longitude
0,1,"West Lafayette, OH",40.2753,-81.751
1,2,"Burr Ridge, IL",41.7489,-87.9184
2,3,"Conway, SC",33.836,-79.0478
3,4,"Keyser, WV",39.4409,-78.9739
4,5,"gressham, OR",45.5045,-122.436
...,...,...,...,...
2126,2127,"Snowflake, AZ",34.5134,-110.078
2127,2128,,,
2128,2129,"El paso, TX",31.7619,-106.485
2129,2130,"CHARLESTOWN, IN",38.4531,-85.6702


In [13]:
# merge the dataframes
final_df = pd.merge(summary,reduced_df,on="ReportNum")

In [14]:
# dropped the empty city and stae rows
clean_df = final_df.dropna(subset=["City","State"])
clean_df

,Date,City,State,Country,Shape,Duration,Summary,Posted,Images,ReportNum,Location,Latitude,Longitude
0,7/31/22 21:59,West Lafayette,OH,USA,Light,4 minutes,Bright circle light moving through sky.,9/9/22,Yes,1,"West Lafayette, OH",40.2753,-81.751
1,7/31/22 21:45,Burr Ridge,IL,USA,Other,NaN,"Strange red, blue and black craft in the sky a...",9/9/22,Yes,2,"Burr Ridge, IL",41.7489,-87.9184
2,7/31/22 21:15,Conway,SC,USA,Circle,Less than a minute,Round bright white light go in a straight line...,9/9/22,NaN,3,"Conway, SC",33.836,-79.0478
3,7/31/22 21:11,Keyser,WV,USA,Light,2 minutes,Saw an orb that interacted with me.,9/9/22,NaN,4,"Keyser, WV",39.4409,-78.9739
4,7/31/22 20:43,gressham,OR,USA,Unknown,less than a minute,A single soft red light that pulsed three time...,9/9/22,NaN,5,"gressham, OR",45.5045,-122.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,12/1/22 18:10,Blakeslee,OH,USA,Sphere,5 minutes,Bright spherical object moving linear west to ...,12/22/22,Yes,2126,"Blakeslee, OH",41.5242,-84.7302
2126,12/1/22 18:00,Snowflake,AZ,USA,Triangle,2 minutes,I grew up on a 40 acre parcel of land with thr...,12/22/22,NaN,2127,"Snowflake, AZ",34.5134,-110.078
2128,12/1/22 14:10,El paso,TX,USA,Oval,Ten minutes,It appeared out of nowhere & slowly moved acro...,12/22/22,NaN,2129,"El paso, TX",31.7619,-106.485
2129,12/1/22 06:20,CHARLESTOWN,IN,USA,Light,2 to 3 minutes,Bright object was seen arriving and silently h...,12/22/22,Yes,2130,"CHARLESTOWN, IN",38.4531,-85.6702


In [19]:
clean_df['Date']=pd.to_datetime(clean_df['Date'])
clean_df.dtypes

Date         datetime64[ns]
City                 object
State                object
Country              object
Shape                object
Duration             object
Summary              object
Posted               object
Images               object
ReportNum             int64
Location             object
Latitude             object
Longitude            object
dtype: object

In [20]:
# write to csv
clean_df.to_csv("UFO_data.csv" ,index=False)